In [2]:
import os
import pandas as pd
import requests
import time

# Ruta al archivo Excel
excel_file = 'f:/RESOLUCIONES GERENCIALES.xlsx'  # Reemplaza con la ruta de tu archivo Excel
# Ruta a la carpeta donde guardarás los PDFs descargados
download_folder = 'D:/Resoluciones'  # Reemplaza con la ruta de tu carpeta de destino

# Crear la carpeta si no existe
if not os.path.exists(download_folder):
    os.makedirs(download_folder)

# Leer el archivo Excel
df = pd.read_excel(excel_file)

# Suponiendo que los enlaces están en la columna 'Enlaces' (ajustar según el nombre real de la columna)
enlaces_columna = 'LINK'  # Cambiar por el nombre de la columna que contiene los enlaces

# Crear una nueva columna para almacenar los nombres de los archivos descargados
df['Nuevo Nombre PDF'] = ''

# Función para convertir URL de Google Drive en un enlace de descarga directa
def convertir_url_drive(url):
    base_url = "https://drive.google.com/uc?export=download&id="
    file_id = url.split('/d/')[1].split('/')[0]
    return base_url + file_id

# Función para descargar y renombrar los PDFs
def descargar_pdf(url, index):
    try:
        # Convertir el enlace de Google Drive en una URL de descarga directa
        url_pdf = convertir_url_drive(url)
        
        # Descargar el archivo PDF
        response = requests.get(url_pdf)
        response.raise_for_status()  # Verificar que la solicitud fue exitosa
        
        # Generar el nuevo nombre con prefijo
        nuevo_nombre = f"doc_{index:02d}.pdf"
        
        # Ruta donde se guardará el archivo
        archivo_path = os.path.join(download_folder, nuevo_nombre)
        
        # Guardar el archivo PDF
        with open(archivo_path, 'wb') as f:
            f.write(response.content)
        
        # Esperar 20 segundos antes de continuar
        time.sleep(20)
        
        return nuevo_nombre
    except Exception as e:
        print(f"Error al descargar el archivo {url}: {e}")
        return None

# Recorrer todas las filas del DataFrame
for idx, row in df.iterrows():
    url_pdf = row[enlaces_columna]
    if isinstance(url_pdf, str) and url_pdf.startswith('https://drive.google.com'):  # Verificar si es un enlace de Google Drive válido
        nuevo_nombre_pdf = descargar_pdf(url_pdf, idx + 1)  # Los índices empiezan en 0, pero prefijo empieza en 01
        if nuevo_nombre_pdf:
            # Agregar el nuevo nombre al DataFrame
            df.at[idx, 'Nuevo Nombre PDF'] = nuevo_nombre_pdf

# Guardar el archivo Excel con la nueva columna
output_excel_file = 'D:/Resoluciones/archivo_modificado.xlsx'  # Define la ruta donde guardar el archivo modificado
df.to_excel(output_excel_file, index=False)

print(f"Archivo Excel modificado guardado en: {output_excel_file}")


Archivo Excel modificado guardado en: D:/Resoluciones/archivo_modificado.xlsx
